In [1]:
import datetime
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
from pandas import Series,DataFrame

data_train = pd.read_csv('train.csv', parse_dates=['date'])
data_test = pd.read_csv('test.csv', parse_dates=['date'])
weather_data = pd.read_csv('weather.csv')

In [2]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14006 entries, 0 to 14005
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      14006 non-null  int64         
 1   date    14006 non-null  datetime64[ns]
 2   speed   14006 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 328.4 KB


In [3]:
def extract_date(df, column):
    df[column+"_year"] = df[column].apply(lambda x: x.year)
    df[column+"_month"] = df[column].apply(lambda x: x.month)
    df[column+"_day"] = df[column].apply(lambda x: x.day)
    df[column+"_hour"] = df[column].apply(lambda x: x.hour)    
    #df[column+"_Spring"] = df[column].apply(lambda x: 1 if ((x.month >= 3) and (x.month <= 5)) else 0)
    #df[column+"_Summer"] = df[column].apply(lambda x: 1 if ((x.month >= 6) and (x.month <= 8)) else 0)
    #df[column+"_Autumn"] = df[column].apply(lambda x: 1 if ((x.month >= 9) and (x.month <= 11)) else 0)
    #df[column+"_Winter"] = df[column].apply(lambda x: 1 if (((x.month >= 1) and (x.month <= 2)) or x.month == 12) else 0)
    #df[column+"_weekofyear"] = df[column].apply(lambda x: x.weekofyear)
    df[column+"_beginofyear"] = df[column].apply(lambda x: 1 if x.month <= 2 else 0)
    df[column+"_endofyear"] = df[column].apply(lambda x: 1 if x.month >= 11 else 0)
    #df[column+"_beginofseason"] = df[column].apply(lambda x: 1 if ((x.month == 3) or (x.month == 6) or (x.month == 9) or (x.month == 12)) else 0)
    #df[column+"_endofseason"] = df[column].apply(lambda x: 1 if ((x.month == 2) or (x.month == 5) or (x.month == 8) or (x.month == 11)) else 0)
    #df[column+"_beginofmonth"] = df[column].apply(lambda x: 1 if x.day <= 5 else 0)
    #df[column+"_endofmonth"] = df[column].apply(lambda x: 1 if x.day >= 25 else 0)
    df[column+"_Mon"] = df[column].apply(lambda x: 1 if x.dayofweek == 0 else 0)
    df[column+"_Tue"] = df[column].apply(lambda x: 1 if x.dayofweek == 1 else 0)
    df[column+"_Wed"] = df[column].apply(lambda x: 1 if x.dayofweek == 2 else 0)
    df[column+"_Thu"] = df[column].apply(lambda x: 1 if x.dayofweek == 3 else 0)
    df[column+"_Fri"] = df[column].apply(lambda x: 1 if x.dayofweek == 4 else 0)
    df[column+"_Sat"] = df[column].apply(lambda x: 1 if x.dayofweek == 5 else 0)
    df[column+"_Sun"] = df[column].apply(lambda x: 1 if x.dayofweek == 6 else 0)
    df[column+"_weekend"] = df[column].apply(lambda x: 1 if((x.dayofweek == 5) or (x.dayofweek == 6)) else 0)
    df[column+"_dawn"] = df[column].apply(lambda x: 1 if ((x.hour >= 2) and (x.hour <= 5)) else 0)
    df[column+"_morning"] = df[column].apply(lambda x: 1 if ((x.hour >= 6) and (x.hour <= 9)) else 0)
    df[column+"_noon"] = df[column].apply(lambda x: 1 if ((x.hour >= 10) and (x.hour <= 13)) else 0)
    df[column+"_afternoon"] = df[column].apply(lambda x: 1 if ((x.hour >= 14) and (x.hour <= 17)) else 0)
    df[column+"_evening"] = df[column].apply(lambda x: 1 if ((x.hour >= 18) and (x.hour <= 21)) else 0)
    df[column+"_midnight"] = df[column].apply(lambda x: 1 if (((x.hour >= 22) and (x.hour <= 23)) or ((x.hour >= 0) and (x.hour <= 1))) else 0)
    df[column+"_holiday"] = df[column].apply(lambda x: 1 \
                                             if(((x.month == 1) and (x.day == 1) and (x.year == 2017)) \
                                                or ((x.month == 1) and (x.day == 2) and (x.year == 2017)) \
                                                or ((x.month == 1) and (x.day == 28) and (x.year == 2017)) \
                                                or ((x.month == 1) and (x.day == 29) and (x.year == 2017)) \
                                                or ((x.month == 1) and (x.day == 30) and (x.year == 2017)) \
                                                or ((x.month == 2) and (x.day == 14) and (x.year == 2017)) \
                                                or ((x.month == 4) and (x.day == 4) and (x.year == 2017))  \
                                                or ((x.month == 4) and (x.day == 14) and (x.year == 2017)) \
                                                or ((x.month == 4) and (x.day == 15) and (x.year == 2017)) \
                                                or ((x.month == 4) and (x.day == 16) and (x.year == 2017)) \
                                                or ((x.month == 4) and (x.day == 17) and (x.year == 2017)) \
                                                or ((x.month == 5) and (x.day == 1) and (x.year == 2017)) \
                                                or ((x.month == 5) and (x.day == 3) and (x.year == 2017)) \
                                                or ((x.month == 5) and (x.day == 14) and (x.year == 2017)) \
                                                or ((x.month == 5) and (x.day == 30) and (x.year == 2017)) \
                                                or ((x.month == 6) and (x.day == 18) and (x.year == 2017)) \
                                                or ((x.month == 7) and (x.day == 1) and (x.year == 2017)) \
                                                or ((x.month == 12) and (x.day == 25) and (x.year == 2017)) \
                                                or ((x.month == 10) and (x.day == 1) and (x.year == 2017)) \
                                                or ((x.month == 10) and (x.day == 2) and (x.year == 2017)) \
                                                or ((x.month == 10) and (x.day == 5) and (x.year == 2017)) \
                                                or ((x.month == 10) and (x.day == 28) and (x.year == 2017)) \
                                                or ((x.month == 12) and (x.day == 26) and (x.year == 2017)) \
                                                or ((x.month == 1) and (x.day == 1) and (x.year == 2018)) \
                                                or ((x.month == 2) and (x.day == 16) and (x.year == 2018)) \
                                                or ((x.month == 2) and (x.day == 17) and (x.year == 2018)) \
                                                or ((x.month == 3) and (x.day == 30) and (x.year == 2018)) \
                                                or ((x.month == 2) and (x.day == 14) and (x.year == 2018)) \
                                                or ((x.month == 3) and (x.day == 31) and (x.year == 2018))  \
                                                or ((x.month == 4) and (x.day == 1) and (x.year == 2018)) \
                                                or ((x.month == 4) and (x.day == 2) and (x.year == 2018)) \
                                                or ((x.month == 4) and (x.day == 5) and (x.year == 2018)) \
                                                or ((x.month == 5) and (x.day == 1) and (x.year == 2018)) \
                                                or ((x.month == 5) and (x.day == 13) and (x.year == 2018)) \
                                                or ((x.month == 5) and (x.day == 22) and (x.year == 2018)) \
                                                or ((x.month == 6) and (x.day == 17) and (x.year == 2018)) \
                                                or ((x.month == 6) and (x.day == 18) and (x.year == 2018)) \
                                                or ((x.month == 7) and (x.day == 1) and (x.year == 2018)) \
                                                or ((x.month == 9) and (x.day == 25) and (x.year == 2018)) \
                                                or ((x.month == 12) and (x.day == 25) and (x.year == 2018)) \
                                                or ((x.month == 10) and (x.day == 1) and (x.year == 2018)) \
                                                or ((x.month == 10) and (x.day == 17) and (x.year == 2018)) \
                                                or ((x.month == 12) and (x.day == 26) and (x.year == 2018))) \
                                             else 0)
    
extract_date(data_train, 'date')
data_train.head(10)

,id,date,speed,date_year,date_month,date_day,date_hour,date_beginofyear,date_endofyear,date_Mon,...,date_Sat,date_Sun,date_weekend,date_dawn,date_morning,date_noon,date_afternoon,date_evening,date_midnight,date_holiday
0,0,2017-01-01 00:00:00,43.002930,2017,1,1,0,1,0,0,...,0,1,1,0,0,0,0,0,1,1
1,1,2017-01-01 01:00:00,46.118696,2017,1,1,1,1,0,0,...,0,1,1,0,0,0,0,0,1,1
2,2,2017-01-01 02:00:00,44.294158,2017,1,1,2,1,0,0,...,0,1,1,1,0,0,0,0,0,1
3,3,2017-01-01 03:00:00,41.067468,2017,1,1,3,1,0,0,...,0,1,1,1,0,0,0,0,0,1
4,4,2017-01-01 04:00:00,46.448653,2017,1,1,4,1,0,0,...,0,1,1,1,0,0,0,0,0,1
5,5,2017-01-01 05:00:00,46.797766,2017,1,1,5,1,0,0,...,0,1,1,1,0,0,0,0,0,1
6,6,2017-01-01 06:00:00,44.404925,2017,1,1,6,1,0,0,...,0,1,1,0,1,0,0,0,0,1
7,7,2017-01-01 07:00:00,45.255897,2017,1,1,7,1,0,0,...,0,1,1,0,1,0,0,0,0,1
8,8,2017-01-01 08:00:00,45.680859,2017,1,1,8,1,0,0,...,0,1,1,0,1,0,0,0,0,1
9,9,2017-01-01 09:00:00,48.435676,2017,1,1,9,1,0,0,...,0,1,1,0,1,0,0,0,0,1


In [4]:
weather_data['date'] = pd.to_datetime(weather_data['date'],format = '%Y/%m/%d').dt.date
data_train['date'] = data_train['date'].dt.date
data_train = pd.merge(data_train, weather_data, on = 'date')

In [5]:
data_train = data_train.drop(["id"], axis = 1).drop(["date"], axis = 1)
data_train.head()

,speed,date_year,date_month,date_day,date_hour,date_beginofyear,date_endofyear,date_Mon,date_Tue,date_Wed,...,Unnamed: 2,temperature,Unnamed: 4,Unnamed: 5,wet,cloud,rain,Unnamed: 9,Unnamed: 10,wind
0,43.002930,2017,1,1,0,1,0,0,0,0,...,20.8,19.2,18.4,15.6,80,72,0.0,4.6,60,34.2
1,46.118696,2017,1,1,1,1,0,0,0,0,...,20.8,19.2,18.4,15.6,80,72,0.0,4.6,60,34.2
2,44.294158,2017,1,1,2,1,0,0,0,0,...,20.8,19.2,18.4,15.6,80,72,0.0,4.6,60,34.2
3,41.067468,2017,1,1,3,1,0,0,0,0,...,20.8,19.2,18.4,15.6,80,72,0.0,4.6,60,34.2
4,46.448653,2017,1,1,4,1,0,0,0,0,...,20.8,19.2,18.4,15.6,80,72,0.0,4.6,60,34.2


In [6]:
from sklearn.model_selection import train_test_split
y = data_train['speed']
features = ['date_year','date_month','date_day','date_hour','date_Mon','date_Tue','date_Wed', \
            'date_Thu','date_Fri','date_Sat','date_Sun','date_weekend','date_holiday',\
            'date_beginofyear','date_endofyear',\
            'date_morning','date_noon','date_afternoon','date_evening','date_midnight','date_dawn',\
           'temperature','wet','cloud','rain','wind']
X = data_train[features].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=14908)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from xgboost.sklearn import XGBRegressor
from lightgbm import LGBMRegressor
import tensorflow as tf 
#model = SGDRegressor(random_state=14908)#0.18
#model = Lasso(random_state=14908)#0.17
#model = RandomForestRegressor(n_estimators=10, random_state=14908)#0.72    52.05
#model = ExtraTreesRegressor(n_estimators=10, random_state=14908)#0.65
#model = GradientBoostingRegressor(n_estimators=10, random_state=0)#0.62
#model = DecisionTreeRegressor(random_state=14908)#0.56
#model = KNeighborsRegressor(4,weights="distance",p=2)#0.78
#model = LGBMRegressor(reg_alpha=0.04,subsample=0.6,max_depth=14,min_child_weight=0,
#                      n_estimators=140,num_leaves=39,learning_rate=0.2)#0.86
#model = XGBRegressor(learning_rate=0.2,max_depth=11, min_child_weight=2,n_estimators=45, subsample=0.8)#14.66
model = XGBRegressor(base_score=0.2,n_estimators=100,max_depth=10,min_child_weight=2,
                     reg_alpha=3.1,subsample=1,learning_rate=0.3,num_parallel_tree=2)
model.fit(X, y)
#model.fit(X_train, y_train)

XGBRegressor(base_score=0.2, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=10,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=2,
             objective='reg:squarederror', random_state=0, reg_alpha=3.1,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [8]:
print(model.score(X_test,y_test))

0.9950233390752101


In [9]:
# Generate test predictions
preds = model.predict(X_test)
print("training accuracy:", (abs(preds - y_test)).sum())

training accuracy: 1997.380322468056


In [97]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    #'base_score':[0.2,0.3,0.4,0.5,0.6],
    #'booster':['gbtree','gblinear'],
    'n_estimators': range(90,150,5)
    #'max_depth': [3, 4, 5, 6, 7, 8, 9, 10], 
    #'min_child_weight': [1, 2, 3, 4, 5, 6],
    #'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
    #'subsample': [0.6, 0.7, 0.8, 0.9,1], 
    #'colsample_bytree': [0.6, 0.7, 0.8, 0.9,1],
    #'reg_alpha': [0.05, 0.1, 1, 2, 3]
    #'reg_lambda': [0.05, 0.1, 1, 2, 3],
    #'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.2, 0.25, 0.30],
    #'max_delta_step': [0,1,2,3]
    #'num_parallel_tree': [1,2,3]
}
gsearch = GridSearchCV(estimator = XGBRegressor(base_score=0.2, booster='gbtree', 
             gpu_id=-1,
             importance_type='gain', 
             learning_rate=0.3,
             n_estimators=100, n_jobs=0, 
             objective='reg:squarederror', random_state=0, reg_alpha=3.1,
             reg_lambda=1, scale_pos_weight=1,
             validate_parameters=1, verbosity=None),param_grid=param_grid,cv=5,scoring='neg_mean_squared_error')
gsearch.fit(X_train,y_train)
print(gsearch.best_params_,gsearch.best_score_)

{'n_estimators': 145} -18.551166001225774


In [10]:
extract_date(data_test, 'date')
data_test['date'] = data_test['date'].dt.date
data_test = pd.merge(data_test, weather_data, on = 'date')
data_test = data_test.drop(["id"], axis = 1).drop(["date"],axis = 1)
data_test.head()

,date_year,date_month,date_day,date_hour,date_beginofyear,date_endofyear,date_Mon,date_Tue,date_Wed,date_Thu,...,Unnamed: 2,temperature,Unnamed: 4,Unnamed: 5,wet,cloud,rain,Unnamed: 9,Unnamed: 10,wind
0,2018,1,1,2,1,0,1,0,0,0,...,19.0,17.3,16.3,13.3,78,75,0.0,3.0,70,28.3
1,2018,1,1,5,1,0,1,0,0,0,...,19.0,17.3,16.3,13.3,78,75,0.0,3.0,70,28.3
2,2018,1,1,7,1,0,1,0,0,0,...,19.0,17.3,16.3,13.3,78,75,0.0,3.0,70,28.3
3,2018,1,1,8,1,0,1,0,0,0,...,19.0,17.3,16.3,13.3,78,75,0.0,3.0,70,28.3
4,2018,1,1,10,1,0,1,0,0,0,...,19.0,17.3,16.3,13.3,78,75,0.0,3.0,70,28.3


In [11]:
features = ['date_year','date_month','date_day','date_hour','date_Mon','date_Tue','date_Wed', \
            'date_Thu','date_Fri','date_Sat','date_Sun','date_weekend','date_holiday',\
            'date_beginofyear','date_endofyear',\
            'date_morning','date_noon','date_afternoon','date_evening','date_midnight','date_dawn',\
           'temperature','wet','cloud','rain','wind']
data_test = data_test[features].copy()
columns_builds = model.get_booster().feature_names
data_test = data_test[columns_builds]
result = model.predict(data_test)
r = pd.DataFrame(result)
r.columns=["speed"]
r.insert(0,'id',r.index)
r["speed"] = r["speed"].astype('float')

In [12]:
r.head()
#r.info()

,id,speed
0,0,48.134258
1,1,48.277649
2,2,35.814003
3,3,23.302536
4,4,36.223431


In [13]:
r.to_csv("result.csv", index=False)